In [8]:
import psycopg2
from psycopg2.extras import execute_values
from multiprocessing import Pool
import time
import threading
import subprocess
import pandas
from pandas import DataFrame

In [4]:
import os
import struct
from resource import *
import resource

In [5]:
print(getrlimit(resource.RLIMIT_DATA))

(-1, -1)


In [6]:
resource.setrlimit(resource.RLIMIT_AS, (4294967296,4294967296)) # 4GB RAM

In [7]:
DB_MAIN_CONFIG = {
    'dbname': 'captivecoredb_history',
    'user': 'captivecore',
    'password': 'stellar123',
    'host': 'localhost',
    'port': 5432
}

DB_FILTERED_CONFIG = {
    'dbname': 'captivecoredb_fil',
    'user': 'jdbc',
    'password': 'stellar123',
    'host': 'localhost',
    'port': 5432
}

In [25]:
conn_main = psycopg2.connect(database="captivecoredb", user="matija", password="stellar123")
cur_main = conn_main.cursor()

In [9]:
conn_his = psycopg2.connect(database="captivecoredb_history", user="matija", password="stellar123")
cur_his = conn_his.cursor()

In [29]:
conn_fil = psycopg2.connect(database="captivecoredb_fil", user="matija", password="stellar123")
cur_fil = conn_fil.cursor()

In [27]:
# Main DB
try:
    cur_main.execute("SELECT count(*), ledger_sequence FROM history_transactions where ledger_sequence between 55403411 and 55406029 group by ledger_sequence order by ledger_sequence asc;")
    #captivecore_transactions = cur_his.fetchone()[0]
    captivecore_transactions_count = DataFrame(cur_main.fetchall())
    print(captivecore_transactions_count)
except Exception as e:
    print(f"Error: {e}")
    conn_main.rollback()  # Rollback on failure

        0         1
0     277  55403411
1     685  55403412
2     290  55403413
3     262  55403414
4     295  55403415
...   ...       ...
2614  192  55406025
2615  224  55406026
2616  191  55406027
2617  203  55406028
2618  170  55406029

[2619 rows x 2 columns]


In [ ]:
# History DB
try:
    cur_his.execute("SELECT count(*), ledger_sequence FROM history_transactions where ledger_sequence between 55403411 and 55406029 group by ledger_sequence order by ledger_sequence asc;")
    #captivecore_transactions = cur_his.fetchone()[0]
    captivecore_transactions_count = DataFrame(cur_his.fetchall())
    print(captivecore_transactions_count)
    print(getrlimit(resource.RLIMIT_DATA))
except Exception as e:
    print(f"Error: {e}")
    conn_his.rollback()  # Rollback on failure

       0         1
0    150  54118601
1    161  54118602
2    196  54118603
3    142  54118604
4    304  54118605
..   ...       ...
995  231  54119596
996  209  54119597
997  236  54119598
998  291  54119599
999  309  54119600

[1000 rows x 2 columns]
(-1, -1)


In [30]:
try:
    cur_fil.execute("SELECT count(*), ledger_sequence FROM transactions where ledger_sequence between 55403411 and 55406029 group by ledger_sequence order by ledger_sequence asc;")
    captivecore_fil_transactions_count = DataFrame(cur_fil.fetchall())
    print(captivecore_fil_transactions_count)
    print(getrlimit(resource.RLIMIT_DATA))
except Exception as e:
    print(f"Error: {e}")
    conn_fil.rollback()  # Rollback on failure
print(getrlimit(resource.RLIMIT_DATA))

        0         1
0     186  55404030
1     312  55404031
2     172  55404032
3     243  55404033
4     199  55404034
...   ...       ...
1895  192  55406025
1896  224  55406026
1897  191  55406027
1898  203  55406028
1899  170  55406029

[1900 rows x 2 columns]
(-1, -1)
(-1, -1)


In [19]:
ledgers = captivecore_transactions_count[1]
ledgers_not_in_filtered_2 = ledgers.to_frame()
#ledgers_not_in_filtered['sequence'] = captivecore_fil_transactions_count[1]
ledgers_not_in_filtered_2['present_in_fil'] = captivecore_transactions_count[1].isin(captivecore_fil_transactions_count[1])
#ledgers_not_in_filtered_2['present_in_main'] = captivecore_fil_transactions_count[1].isin(captivecore_transactions_count[1])

In [24]:
captivecore_transactions_count['not_ingested_transactions'] = captivecore_transactions_count[0] - captivecore_fil_transactions_count[0]

In [17]:
cur_his.execute("TRUNCATE history_ledgers;")


In [27]:
try:
    cur1.execute("SELECT id, ledger_sequence FROM history_transactions where ledger_sequence = 54823868 or ledger_sequence = 54639529;")
    #captivecore_transactions = cur1.fetchone()[0]
    captivecore_transactions_54479614_54394229 = DataFrame(cur1.fetchall())
    print(captivecore_transactions_54479614_54394229)
except Exception as e:
    print(f"Error: {e}")
    conn1.rollback()  # Rollback on failure

                      0         1
0    234674990124244992  54639529
1    234674990124605440  54639529
2    234674990123868160  54639529
3    234674990123872256  54639529
4    234674990123929600  54639529
..                  ...       ...
467  235466720101093376  54823868
468  235466720101097472  54823868
469  235466720101101568  54823868
470  235466720101105664  54823868
471  235466720101109760  54823868

[472 rows x 2 columns]


In [28]:
try:
    cur2.execute("SELECT id, ledger_sequence FROM transactions where ledger_sequence = 54823868 or ledger_sequence = 54639529;")
    #captivecore_transactions = cur1.fetchone()[0]
    captivecore_fil_transactions_54479614_54394229 = DataFrame(cur2.fetchall())
    print(captivecore_fil_transactions_54479614_54394229)
except Exception as e:
    print(f"Error: {e}")
    conn2.rollback()  # Rollback on failure

                      0         1
0    234674990123847680  54639529
1    234674990123851776  54639529
2    234674990123855872  54639529
3    234674990123859968  54639529
4    234674990123864064  54639529
..                  ...       ...
464  235466720101093376  54823868
465  235466720101097472  54823868
466  235466720101101568  54823868
467  235466720101105664  54823868
468  235466720101109760  54823868

[469 rows x 2 columns]


In [29]:
captivecore_transactions_54479614_54394229['present_in_fil'] = captivecore_transactions_54479614_54394229[0].isin(captivecore_fil_transactions_54479614_54394229[0])

In [33]:
try:
    cur1.execute("SELECT count(*) FROM history_ledgers where sequence between 54284800 and 55173365;")
    captivecore_ledgers = cur1.fetchone()[0]
    #captivecore_transactions_count = DataFrame(cur1.fetchall())
    print(captivecore_ledgers)
except Exception as e:
    print(f"Error: {e}")
    conn1.rollback()  # Rollback on failure

888566


In [34]:
try:
    cur2.execute("SELECT count(*) FROM ledgers where sequence between 54284800 and 55173365;")
    captivecore_fil_ledgers = cur2.fetchone()[0]
    #captivecore_transactions_count = DataFrame(cur1.fetchall())
    print(captivecore_fil_ledgers)
except Exception as e:
    print(f"Error: {e}")
    conn2.rollback()  # Rollback on failure

888566


In [4]:
import os
import struct
import time

def find_offset_in_log(file_path, target_offset):
    with open(file_path, 'rb') as log_file:

        # The log files are organized in batches of messages
        while True:
            # Read the next batch of messages (4 bytes to indicate the length of the log entry)
            length_bytes = log_file.read(4)
            if len(length_bytes) < 4:
                break  # End of file reached or incomplete entry

            # Decode the length of the log entry (int)
            length = struct.unpack('>I', length_bytes)[0]

            # Read the log entry data
            log_data = log_file.read(length)
            if len(log_data) < length:
                break  # Incomplete log data, break out

            # Check if we have enough data to unpack the offset (8 bytes)
            for i in range(len(log_data) - 7):  # Ensure we have 8 bytes to check
                possible_offset = struct.unpack('>Q', log_data[i:i+8])[0]

                # Check if we found the target offset
                if possible_offset == target_offset:
                    print(f"Found message with offset {target_offset} starting at byte {i} in log data.")
                    print(f"Log data: {log_data[i:i+8]}")
                    return True  # Found the message

        print(f"Offset {target_offset} not found in log file.")
        return False  # Offset not found

def get_file_creation_time(file_path):
    # Get the last modified time of the file
    modified_time = os.path.getmtime(file_path)
    # Convert it to a human-readable format
    return time.ctime(modified_time)

# Example usage
if __name__ == "__main__":
    log_file_path = '/home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/00000000000004773221.log'  # Replace with your log file path
    print(f"Log file was last modified at: {get_file_creation_time(log_file_path)}")

    target_offset = 779523771  # Replace with your target offset

    if find_offset_in_log(log_file_path, target_offset):
        print(f"Offset {target_offset} found.")
    else:
        print(f"Offset {target_offset} not found.")


Log file was last modified at: Tue Jan 28 19:40:26 2025
Offset 779523771 not found in log file.
Offset 779523771 not found.


In [6]:
log_file_path = '/home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/00000000000002386165.log'  # Replace with your log file path
print(f"Log file was last modified at: {get_file_creation_time(log_file_path)}")

Log file was last modified at: Tue Jan 28 19:37:55 2025


In [35]:
log_file_path = '/home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/00000000000652738148.log'  # Replace with your log file path
print(f"Log file was last modified at: {get_file_creation_time(log_file_path)}")


Log file was last modified at: Mon Jan 27 15:52:23 2025


In [37]:
log_file_path = '/home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/00000000000905953845.log'  # Replace with your log file path
print(f"Log file was last modified at: {get_file_creation_time(log_file_path)}")

Log file was last modified at: Tue Jan 28 15:00:12 2025


In [34]:
import datetime

now = datetime.datetime.now()
print(now)

2025-01-28 14:16:07.642794


In [19]:
import os
import struct
import lz4.frame

def find_offset_in_log(file_path, target_offset):
    with open(file_path, 'rb') as log_file:
        decompressed_data = lz4.frame.decompress(log_file.read())
        print(decompressed_data[:100])
        # Decompress the LZ4-compressed data
        with lz4.frame.open(log_file, mode='rb') as decompressed_file:
            while True:
                # Read the next 4 bytes to get the length of the log entry
                length_bytes = decompressed_file.read(4)
                if len(length_bytes) < 4:
                    break  # End of file reached or incomplete entry

                # Decode the length of the log entry (int)
                length = struct.unpack('>I', length_bytes)[0]

                # Read the log entry data based on the decoded length
                log_data = decompressed_file.read(length)
                if len(log_data) < length:
                    break  # Incomplete log data, break out

                # Check if we have enough data to unpack the offset (8 bytes)
                if len(log_data) >= 8:
                    # The first 8 bytes of the log entry represent the offset
                    message_offset = struct.unpack('>Q', log_data[:8])[0]

                    # Debugging: print the current offset and the target offset
                    print(f"Current offset: {message_offset}, Target offset: {target_offset}")

                    # Check if the message offset matches the target offset
                    if message_offset == target_offset:
                        print(f"Found message with offset {target_offset} in {file_path}: {log_data}")
                        return True  # Found the message

        return False  # Offset not found in this log file

def search_in_logs(directory, target_offset):
    # Loop through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.log'):
            file_path = os.path.join(directory, filename)
            print(f"Searching in {file_path}...")
            if find_offset_in_log(file_path, target_offset):
                print(f"Offset {target_offset} found in {file_path}.")
                return True  # Stop searching once we find the offset
    print(f"Offset {target_offset} not found in any log files.")
    return False  # If not found in any files

# Example usage
if __name__ == "__main__":
    log_directory = '/home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/'  # Replace with your log directory path
    target_offset = 779523771  # Replace with your target offset

    search_in_logs(log_directory, target_offset)


Searching in /home/matija/kafka_2.13-3.8.0/logs/server/captivecore.public.history_transactions-0/00000000000862923084.log...


RuntimeError: LZ4F_getFrameInfo failed with code: ERROR_frameType_unknown